In [1]:
import nltk
import pandas as pd
import os

# Propositional Logic

In [2]:
# assign symbols and propositions
symbol_P = 'P'
symbol_Q = 'Q'

proposition_P = 'He is hungry'
propositon_Q = 'He will eat a sandwich'

In [3]:
# assign various truth values to the propositions
p_statuses = [False, False, True, True]
q_statuses = [False, True, False, True]

In [4]:
# assign the various expressions combining the logical operators
conjunction = '(P & Q)'
disjunction = '(P | Q)'
implication = '(P -> Q)'
equivalence = '(P <-> Q)'
expressions = [conjunction, disjunction, implication, equivalence]
expressions

['(P & Q)', '(P | Q)', '(P -> Q)', '(P <-> Q)']

In [5]:
# evaluate each expression using propositional logic
results = []
for status_p, status_q in zip(p_statuses, q_statuses):
    dom = set([])
    val = nltk.Valuation([(symbol_P, status_p), 
                          (symbol_Q, status_q)])
    assignments = nltk.Assignment(dom)
    model = nltk.Model(dom, val)
    row = [status_p, status_q]
    for expression in expressions:
    # evaluate each expression based on proposition truth values
        result = model.evaluate(expression, assignments) 
        row.append(result)
    results.append(row)

In [6]:
# build the result table
columns = [symbol_P, symbol_Q, conjunction, 
           disjunction, implication, equivalence]           
result_frame = pd.DataFrame(results, columns=columns)

In [7]:
# display results
print('P:', proposition_P)
print('Q:', propositon_Q)
print()
print('Expression Outcomes:-')
print(result_frame)   

P: He is hungry
Q: He will eat a sandwich

Expression Outcomes:-
       P      Q  (P & Q)  (P | Q)  (P -> Q)  (P <-> Q)
0  False  False    False    False      True       True
1  False   True    False     True      True      False
2   True  False    False     True     False      False
3   True   True     True     True      True       True


# First Order Logic

In [8]:
import nltk
import os

# for reading FOL expressions
read_expr = nltk.sem.Expression.fromstring

# initialize theorem provers (you can choose any)
os.environ['PROVER9'] = r'E:/prover9/bin'
prover = nltk.Prover9()

# I use the following one for our examples
prover = nltk.ResolutionProver() 
prover

In [9]:
# set the rule expression
rule = read_expr('all x. all y. (jumps_over(x, y) -> -jumps_over(y, x))')

# set the event occured
event = read_expr('jumps_over(fox, dog)')

# set the outcome we want to evaluate -- the goal
test_outcome = read_expr('jumps_over(dog, fox)')

# get the result
prover.prove(goal=test_outcome, 
             assumptions=[event, rule],
             verbose=True)

[1] {-jumps_over(dog,fox)}                    A 
[2] {jumps_over(fox,dog)}                     A 
[3] {-jumps_over(z4,z3), -jumps_over(z3,z4)}  A 
[4] {-jumps_over(dog,fox)}                    (2, 3) 



False

In [11]:
# set the rule expression                          
rule = read_expr('all x. (studies(x, exam) -> pass(x, exam))') 

# set the events and outcomes we want to determine
event1 = read_expr('-studies(John, exam)')  
test_outcome1 = read_expr('pass(John, exam)') 

prover.prove(goal=test_outcome1, 
             assumptions=[event1, rule],
             verbose=True)  

[1] {-pass(John,exam)}                  A 
[2] {-studies(John,exam)}               A 
[3] {-studies(z6,exam), pass(z6,exam)}  A 
[4] {-studies(John,exam)}               (1, 3) 



False

In [12]:
event2 = read_expr('studies(Pierre, exam)')  
test_outcome2 = read_expr('pass(Pierre, exam)') 

prover.prove(goal=test_outcome2, 
             assumptions=[event2, rule],
             verbose=True) 

[1] {-pass(Pierre,exam)}                A 
[2] {studies(Pierre,exam)}              A 
[3] {-studies(z8,exam), pass(z8,exam)}  A 
[4] {-studies(Pierre,exam)}             (1, 3) 
[5] {pass(Pierre,exam)}                 (2, 3) 
[6] {}                                  (1, 5) 



True

In [13]:
# define symbols (entities\functions) and their values
rules = """
    rover => r
    felix => f
    garfield => g
    alex => a
    dog => {r, a}
    cat => {g}
    fox => {f}
    runs => {a, f}
    sleeps => {r, g}
    jumps_over => {(f, g), (a, g), (f, r), (a, r)}
    """

val = nltk.Valuation.fromstring(rules)
val

{'rover': 'r',
 'felix': 'f',
 'garfield': 'g',
 'alex': 'a',
 'dog': {('a',), ('r',)},
 'cat': {('g',)},
 'fox': {('f',)},
 'runs': {('a',), ('f',)},
 'sleeps': {('g',), ('r',)},
 'jumps_over': {('a', 'g'), ('a', 'r'), ('f', 'g'), ('f', 'r')}}

In [14]:
dom = {'r', 'f', 'g', 'a'}
m = nltk.Model(dom, val)
m

({'g', 'a', 'f', 'r'}, {'rover': 'r', 'felix': 'f', 'garfield': 'g', 'alex': 'a', 'dog': {('r',), ('a',)}, 'cat': {('g',)}, 'fox': {('f',)}, 'runs': {('a',), ('f',)}, 'sleeps': {('r',), ('g',)}, 'jumps_over': {('f', 'g'), ('a', 'g'), ('f', 'r'), ('a', 'r')}})

In [15]:
m.evaluate('jumps_over(felix, rover) & dog(rover) & runs(rover)', None)

False

In [16]:
m.evaluate('jumps_over(felix, rover) & dog(rover) & -runs(rover)', None)

True

In [17]:
m.evaluate('jumps_over(alex, garfield) & dog(alex) & cat(garfield) & sleeps(garfield)', None)

True

In [18]:
g = nltk.Assignment(dom, [('x', 'r'), ('y', 'f')])   
g

{'x': 'r', 'y': 'f'}

In [19]:
m.evaluate('runs(y) & jumps_over(y, x) & sleeps(x)', g)   

True

In [20]:
m.evaluate('exists y. (fox(y) & runs(y))', g)

True

In [21]:
formula = read_expr('runs(x)')
m.satisfiers(formula, 'x', g)

{'a', 'f'}

In [22]:
formula = read_expr('runs(x) & fox(x)')
m.satisfiers(formula, 'x', g)

{'f'}